In [1]:
pip install pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import sum, avg, max, min, count

In [4]:
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
df=spark.read.csv("/content/covid_19_clean_complete.csv",header=True,inferSchema=True)

In [6]:
df.show()

+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|      Province/State|     Country/Region|      Lat|      Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|
+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|                NULL|        Afghanistan| 33.93911| 67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|
|                NULL|            Albania|  41.1533|   20.1683|2020-01-22|        0|     0|        0|     0|              Europe|
|                NULL|            Algeria|  28.0339|    1.6596|2020-01-22|        0|     0|        0|     0|              Africa|
|                NULL|            Andorra|  42.5063|    1.5218|2020-01-22|        0|     0|        0|     0|              Europe|
|                NULL|             Angola| -11.2027|   17.8739|2020-01-22|        0|     0

In [7]:
df.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)



In [8]:
dfchina = df.filter(df["Country/Region"]=="China")
dfchina.show()

+--------------+--------------+------------------+--------+----------+---------+------+---------+------+---------------+
|Province/State|Country/Region|               Lat|    Long|      Date|Confirmed|Deaths|Recovered|Active|     WHO Region|
+--------------+--------------+------------------+--------+----------+---------+------+---------+------+---------------+
|         Anhui|         China|           31.8257|117.2264|2020-01-22|        1|     0|        0|     1|Western Pacific|
|       Beijing|         China|           40.1824|116.4142|2020-01-22|       14|     0|        0|    14|Western Pacific|
|     Chongqing|         China|           30.0572| 107.874|2020-01-22|        6|     0|        0|     6|Western Pacific|
|        Fujian|         China|           26.0789|117.9874|2020-01-22|        1|     0|        0|     1|Western Pacific|
|         Gansu|         China|           35.7518|104.2861|2020-01-22|        0|     0|        0|     0|Western Pacific|
|     Guangdong|         China| 

In [9]:
df1= df.filter(df.Deaths>100)
df1.show()

+--------------+--------------+-------+--------+----------+---------+------+---------+------+---------------+
|Province/State|Country/Region|    Lat|    Long|      Date|Confirmed|Deaths|Recovered|Active|     WHO Region|
+--------------+--------------+-------+--------+----------+---------+------+---------+------+---------------+
|         Hubei|         China|30.9756|112.2707|2020-01-28|     3554|   125|       80|  3349|Western Pacific|
|         Hubei|         China|30.9756|112.2707|2020-01-29|     3554|   125|       88|  3341|Western Pacific|
|         Hubei|         China|30.9756|112.2707|2020-01-30|     4903|   162|       90|  4651|Western Pacific|
|         Hubei|         China|30.9756|112.2707|2020-01-31|     5806|   204|      141|  5461|Western Pacific|
|         Hubei|         China|30.9756|112.2707|2020-02-01|     7153|   249|      168|  6736|Western Pacific|
|         Hubei|         China|30.9756|112.2707|2020-02-02|    11177|   350|      295| 10532|Western Pacific|
|         

In [10]:
df2=df.groupBy("Country/Region").agg(sum("Deaths").alias("TotalDeaths"))
df2.show()

+--------------+-----------+
|Country/Region|TotalDeaths|
+--------------+-----------+
|          Chad|       5523|
|      Paraguay|       1663|
|        Russia|     619385|
|         Yemen|      17707|
|       Senegal|       7177|
|    Cabo Verde|        854|
|        Sweden|     448913|
|        Guyana|       1346|
|       Eritrea|          0|
|   Philippines|     110892|
|         Burma|        639|
|      Djibouti|       3011|
|      Malaysia|      12971|
|     Singapore|       2441|
|          Fiji|          0|
|        Turkey|     466056|
|        Malawi|       1640|
|Western Sahara|         63|
|          Iraq|     121392|
|       Germany|     871322|
+--------------+-----------+
only showing top 20 rows



In [11]:
df2.orderBy(df2["TotalDeaths"].desc()).show()

+--------------+-----------+
|Country/Region|TotalDeaths|
+--------------+-----------+
|            US|   11011411|
|United Kingdom|    3997775|
|        Brazil|    3938034|
|         Italy|    3707717|
|        France|    3048524|
|         Spain|    3033030|
|        Mexico|    1728277|
|         India|    1111831|
|          Iran|    1024136|
|       Belgium|     963679|
|       Germany|     871322|
|        Canada|     699566|
|         China|     672413|
|          Peru|     652113|
|   Netherlands|     622314|
|        Russia|     619385|
|        Turkey|     466056|
|        Sweden|     448913|
|       Ecuador|     346618|
|         Chile|     322480|
+--------------+-----------+
only showing top 20 rows



In [12]:
df2.write.csv("output/summary", header=True)

In [13]:
dffinal=spark.read.csv("output/summary",header=True,inferSchema=True)
dffinal.show()

+--------------+-----------+
|Country/Region|TotalDeaths|
+--------------+-----------+
|          Chad|       5523|
|      Paraguay|       1663|
|        Russia|     619385|
|         Yemen|      17707|
|       Senegal|       7177|
|    Cabo Verde|        854|
|        Sweden|     448913|
|        Guyana|       1346|
|       Eritrea|          0|
|   Philippines|     110892|
|         Burma|        639|
|      Djibouti|       3011|
|      Malaysia|      12971|
|     Singapore|       2441|
|          Fiji|          0|
|        Turkey|     466056|
|        Malawi|       1640|
|Western Sahara|         63|
|          Iraq|     121392|
|       Germany|     871322|
+--------------+-----------+
only showing top 20 rows

